# tf (transform) in ROS - time and frame (2018/10/19)

Following by http://wiki.ros.org/tf/Tutorials 4~5

base on last log - *20181017_Log ( tf_training )*, mainly modify turtle_tf_listener.cpp in this log.

---
  In last log, we learned about how tf keep track of tree of coordinate frames, wihle tree change over time, tf stores a time snapshhot for every transform (for up to 10 seconds by default). And then we use  **tf::TransformListener::lookupTransfor()** to access the latest available transforms in that tf tree(by **ros::Time(0)** ). 
    
  However, we are going to learn how to get a transform at **specific time** in tis log.
  
---

---

### 1. ros::Time() v.s. ros::Time::now()  

`$ roscd learning_tf`  and open *turtle_tf_listener.cpp*, we can see this section:

---
```
   try{
      listener.lookupTransform("/turtle2", "/carrot1",ros::Time(0), transform);
```
---
  note: **ros::Time(0)** means "the latest available" transform in te buffer

  however,we replace it by **ros:::Time::now()**, and do  
  `$ catkin_make`    
  `$ roslaunch learning_ft start_demo.launch`    
  (don't forget to source your setup.bash file before that)
  
  error msg (keep error):  
  `[ERROR] [1539931257.462804426]: Lookup would require extrapolation into the future.  Requested time 1539931257.462622383 but the latest data is at time 1539931257.453638201, when looking up transform from frame [turtle1] to frame [turtle2]`

  The error msgs is because when a broadcaster send out a transform , it take some time before that transform gets into the buffer. In this case, when we request a frame transform at time "now", the transform we request have not exit in the buffer yet, therefore, we sould wait a few milliseconds.
  
---  
### 2. wait for transform ( *ros::Duration()* )

modify turtle_tf_listener.cpp :

```
  try{
    ros::Time now = ros::Time::now();
    listener.waitForTransform("/turtle2", "/turtle1", now, ros::Duration(3.0));
    listener.lookupTransform("/turtle2", "/turtle1", now, transform);
```
**tf::TransformListener::waitForTransform("target_frame_id", "source_frame_id", at_this_time, timeout)**   :  
1. **waitForTransform** will block until two frame becomes available
2. or if te transform does not available until timeout  
  
However, it will still has an error msgs in thhe beginning :  
`[ERROR] [1539932260.705257034]: Lookup would require extrapolation into the past.  Requested time 1539932257.693989909 but the earliest data is at time 1539932258.055019676, when looking up transform from frame [turtle1] to frame [turtle2]
`

The error reason is that turtle2 takes a non-zero time to spawn, and publish after that, so the first time we request a transform not exist, othherwise, it will run well after turtle2 spawn.

Till now, we have already learned the different between Time(0) and now(), however, for real tf use cases, it is often fine to use Time(0)

---

### 3. track past frame

modify turtle_tf_listener.cpp :
```
  try{
    ros::Time past = ros::Time::now() - ros::Duration(5.0);
    listener.waitForTransform("/turtle2", "/turtle1", past, ros::Duration(1.0));
    listener.lookupTransform("/turtle2", "/turtle1", past, transform);
```

do catkin_make and roslaunch as usually , turtle2 will run a messy route (and my terminal crash), the reason is that we are requesting "**transform of 'pose of turtle1 5 seconds ago' relative to 'pose if turtle2 5 seconds ago'**", and control turtle2 pose according to this WTF transform. 

However, our ideal request should be "**transform of 'pose of turtle1 5 seconds ago' relative to 'pose of turtle2 now'**",but how? 

---

### 4. different parametes form of waitForTransform and lookupTransform

modify turtle_tf_listener.cpp :  

```
 try{
    ros::Time now = ros::Time::now();
    ros::Time past = now - ros::Duration(5.0);
    listener.waitForTransform("/turtle2", now, "/turtle1", past, "/world", ros::Duration(1.0));
    listener.lookupTransform("/turtle2", now, "/turtle1", past, "/world", transform);
```

In **tf::Transformer Class Reference** (https://goo.gl/TKUAHs), waitForTransform and lookupTransform have different parameter form:

```
void Transformer::lookupTransform 	
( 	
    const std::string &  	target_frame,
    const ros::Time &  	    target_time,
    const std::string &  	source_frame,
    const ros::Time &  	    source_time,
    const std::string &  	fixed_frame,
    StampedTransform &  	transform 
	) 		const
```
this parameter form can cited the time of frame we want to track, and provide a fixed_frame which treat the transform as constant in time, here is /world.

```
bool Transformer::waitForTransform 	
( 	
    const std::string &  	target_frame,
    const ros::Time &   	target_time,
	const std::string &  	source_frame,
	const ros::Time &  	    source_time,
	const std::string &  	fixed_frame,
	const ros::Duration &  	timeout,
	const ros::Duration &  	polling_sleep_duration = ros::Duration(0.01),
	std::string *  	error_msg = NULL 
	) 		const
```
mostly like lookupTransform,  polling_sleep_duration means "How often to retest if failed".

---

Finally, tuetle2 can direct to where turtle1 was 5 seconds ago

Tutorial of tf is finis now let's go to study rviz now!

    